1. Вам нужно будет загрузить и оформить датасет котиков, собак https://www.kaggle.com/andrewmvd/dog-and-cat-detection
2. *Нужно будет произвести аугментацию данных

3. Затем обучить нейронную сеть на получившемся датасете

In [282]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /private/var/folders/yv/c8pj7ptn37x0f2cy3vcj2gyh0000gn/T/pip-req-build-wjrh6bhz
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /private/var/folders/yv/c8pj7ptn37x0f2cy3vcj2gyh0000gn/T/pip-req-build-wjrh6bhz
  Created wheel for pycocotools: filename=pycocotools-2.0-cp38-cp38-macosx_10_9_x86_64.whl size=89067 sha256=8fb66766de00e25897aa4596741c4942c264ece7465dae0ad74055e339b304d2
  Stored in directory: /private/var/folders/yv/c8pj7ptn37x0f2cy3vcj2gyh0000gn/T/pip-ephem-wheel-cache-jmqw3zfm/wheels/56/da/49/cb71a7c450b59588934077f431100c05fbde50646ee84a8d40
Successfully built pycocotools


In [283]:
# Download TorchVision repo to use some files from
# references/detection
!git clone https://github.com/pytorch/vision.git
!git checkout v0.3.0

!cp ./vision/references/detection/utils.py ./
!cp ./vision/references/detection/transforms.py ./
!cp ./vision/references/detection/coco_eval.py ./
!cp ./vision/references/detection/engine.py ./
!cp ./vision/references/detection/coco_utils.py ./

fatal: destination path 'vision' already exists and is not an empty directory.
error: pathspec 'v0.3.0' did not match any file(s) known to git


In [296]:
import os

import numpy as np
import pandas as pd

import cv2

from PIL import Image, ImageDraw
from seaborn import color_palette

from pathlib import Path
from xml.etree import ElementTree

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Subset

import torchvision

from engine import train_one_epoch, evaluate
import utils
import transforms as T

In [3]:
images_path = '/Users/macbook/Downloads/archive-2/images'
annotations_path = '/Users/macbook/Downloads/archive-2/annotations'

In [154]:
def parse_xml(files_path):
    
    annotations_list = []
    
    for path in files_path:
        
        annotation_dict = {}
        
        root = ElementTree.parse(path).getroot()
        
        annotation_dict['filename'] = Path(images_path + '/' + root.find('./filename').text)
        
        annotation_dict['width'] = int(root.find('./size/width').text)
        annotation_dict['height'] = int(root.find('./size/height').text)
        
        annotation_dict['class'] = root.find('./object/name').text
        
        annotation_dict['xmin'] = int(root.find('./object/bndbox/xmin').text)
        annotation_dict['ymin'] = int(root.find('./object/bndbox/ymin').text)
        annotation_dict['xmax'] = int(root.find('./object/bndbox/xmax').text)
        annotation_dict['ymax'] = int(root.find('./object/bndbox/ymax').text)
        
        annotations_list.append(annotation_dict)
    
    dataframe = pd.DataFrame(annotations_list)
    
    return dataframe

In [155]:
df = parse_xml(list(Path(annotations_path).glob('*.xml')))

In [156]:
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,/Users/macbook/Downloads/archive-2/images/Cats...,500,333,dog,10,5,457,305
1,/Users/macbook/Downloads/archive-2/images/Cats...,375,500,cat,102,15,272,204
2,/Users/macbook/Downloads/archive-2/images/Cats...,300,225,dog,84,31,216,171
3,/Users/macbook/Downloads/archive-2/images/Cats...,375,500,dog,160,59,256,179
4,/Users/macbook/Downloads/archive-2/images/Cats...,500,332,cat,250,40,486,242


In [35]:
vc = df['class'].value_counts()
vc

dog    2498
cat    1188
Name: class, dtype: int64

In [219]:
classes_dict = dict(zip(vc.keys().to_list(), np.arange(vc.shape[0])))
classes_dict

{'dog': 0, 'cat': 1}

In [221]:
class_num_to_name = dict(zip(np.arange(vc.shape[0]), vc.keys().to_list()))
class_num_to_name

{0: 'dog', 1: 'cat'}

In [224]:
df['class'] = df['class'].apply(lambda x: classes_dict[x])

In [228]:
df['class'] = df['class'].astype(np.uint8)

In [229]:
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,new_path,new_bb
0,/Users/macbook/Downloads/archive-2/images/Cats...,500,333,0,10,5,457,305,resized_images/Cats_Test2531.png,"[6.0, 4.0, 274.0, 275.0]"
1,/Users/macbook/Downloads/archive-2/images/Cats...,375,500,1,102,15,272,204,resized_images/Cats_Test1038.png,"[81.0, 9.0, 218.0, 122.0]"
2,/Users/macbook/Downloads/archive-2/images/Cats...,300,225,0,84,31,216,171,resized_images/Cats_Test134.png,"[84.0, 41.0, 216.0, 229.0]"
3,/Users/macbook/Downloads/archive-2/images/Cats...,375,500,0,160,59,256,179,resized_images/Cats_Test1986.png,"[128.0, 35.0, 205.0, 107.0]"
4,/Users/macbook/Downloads/archive-2/images/Cats...,500,332,1,250,40,486,242,resized_images/Cats_Test652.png,"[150.0, 36.0, 291.0, 219.0]"


In [161]:
def conver_an_image_colo_to_rgb(image):
    
    change_color_from_to = cv2.COLOR_BGR2RGB
    converted_colors_image = cv2.cvtColor(image, change_color_from_to)
    
    return converted_colors_image

def read_image(image_path):
    
    image = cv2.imread(str(image_path))
    
    image = conver_an_image_colo_to_rgb(image)
    
    return image

def bb2mask(im, bb):
    
    """Конвертируем bounding box в маску"""
    
    mask = np.zeros(im.shape[:-1])
    bb = bb.astype(np.int)
    x_min, y_min, x_max, y_max = bb
    mask[y_min:y_max + 1, x_min:x_max + 1] = 1
    
    return mask

def mask2bb(mask):
    
    """Конвертируем маску в bounding box, принимая 0 как фоновый ненулевой объект"""
    
    cols, rows = np.nonzero(mask)
    
    if len(cols) == 0: 
        return np.zeros(4, dtype=np.float32)
    
    x_min = np.min(rows)
    y_min = np.min(cols)
    x_max = np.max(rows)
    y_max = np.max(cols)
    
    return np.array([x_min, y_min, x_max, y_max], dtype=np.float32)

In [163]:
def resize_an_image(image_path, write_path, bb, new_size):
    
    image = read_image(image_path)
    mask = bb2mask(image, bb)
    
    resized_image = cv2.resize(image, (new_size, new_size))
    resized_mask = cv2.resize(mask, (new_size, new_size))
    
    new_image_path = Path(str(write_path) + '/' + image_path.parts[-1])
    
    cv2.imwrite(str(new_image_path), conver_an_image_colo_to_rgb(resized_image))
    
    new_bb = mask2bb(resized_mask)
    
    return new_image_path, new_bb

In [257]:
def draw_box(image_path, bb, cl, cl_dict):
    
    color = (np.array(color_palette('hls', 1)) * 255).astype(np.uint8)[0]
    
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    draw.rectangle(bb, outline=tuple(color), width=3)
    
    text = f"{cl_dict[cl].upper()}"
    text_size = draw.textsize(text)
    
    text_x, text_y = text_size
    
    x_min, y_min = bb[:2]
    
    text_box_xy = [x_min, y_min + text_y, x_min + text_x, y_min]
    text_xy = [x_min, y_min]
    
    draw.rectangle(text_box_xy, fill=tuple(color))
    draw.text(text_xy, text, fill='white')
    
    return image

In [152]:
resized_images_folder = Path('resized_images')

image_path_column = 'filename'
bb_columns = ['xmin', 'ymin', 'xmax', 'ymax']

IMG_SIZE = 300

In [158]:
def add_to_df_new_cols(x, write_path, new_size):

    image_path = x[image_path_column]
    bb = x[bb_columns].values
    
    new_path, new_bb = resize_an_image(image_path, write_path, bb, new_size)
    
    x['new_path'] = new_path
    x['new_bb'] = new_bb
    
    return x

In [167]:
Path.mkdir(resized_images_folder, exist_ok=True)

In [164]:
df = df.apply(lambda x: add_to_df_new_cols(x, resized_images_folder, IMG_SIZE), axis=1)

In [168]:
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax,new_path,new_bb
0,/Users/macbook/Downloads/archive-2/images/Cats...,500,333,dog,10,5,457,305,resized_images/Cats_Test2531.png,"[6.0, 4.0, 274.0, 275.0]"
1,/Users/macbook/Downloads/archive-2/images/Cats...,375,500,cat,102,15,272,204,resized_images/Cats_Test1038.png,"[81.0, 9.0, 218.0, 122.0]"
2,/Users/macbook/Downloads/archive-2/images/Cats...,300,225,dog,84,31,216,171,resized_images/Cats_Test134.png,"[84.0, 41.0, 216.0, 229.0]"
3,/Users/macbook/Downloads/archive-2/images/Cats...,375,500,dog,160,59,256,179,resized_images/Cats_Test1986.png,"[128.0, 35.0, 205.0, 107.0]"
4,/Users/macbook/Downloads/archive-2/images/Cats...,500,332,cat,250,40,486,242,resized_images/Cats_Test652.png,"[150.0, 36.0, 291.0, 219.0]"


In [43]:
df = df.reset_index()

In [169]:
X = df[['new_path', 'new_bb']]
y = df['class']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [264]:
def image_normalize(image):
    imagenet_stats = np.array([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])
    
    image -= imagenet_stats[0]
    image /= imagenet_stats[1]
    
    return image

In [285]:
class CatDogDataset(Dataset):
    
    def __init__(self, paths, bbs, classes, transforms=None):
        self.paths = paths.values
        self.bbs = bbs.values
        self.classes = classes.values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        
        path = self.paths[idx]
        y_class = self.classes[idx]
        box = self.bbs[idx]
        box = torch.as_tensor([box], dtype=torch.float32)
        
        img = Image.open(path)
        image_id = torch.tensor([idx])
        labels = torch.as_tensor([y_class])
        area = (box[:, 3] - box[:, 1]) * (box[:, 2] - box[:, 0])
        iscrowd = torch.tensor([0])
        
        target = {}
        target["boxes"] = box
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms:
            img, target = self.transforms(img, target)
        
        return img, target

In [297]:
class CatsDogsNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(8)
        
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(16)
        
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.bn3 = nn.BatchNorm2d(32)
        
        self.classifier = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 2))
        self.bb = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 2))
    
    def forward(self, x):
        
        print(x.shape)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        print(x.shape)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        print(x.shape)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)
        print(x.shape)
        return self.classifier(x), self.bb(x)

In [288]:
def get_transform(train: bool):
    transforms = []
    
    transforms.append(T.ToTensor())
    
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        
    return T.Compose(transforms)

In [291]:
dataset_train = CatDogDataset(df['new_path'], df['new_bb'], df['class'], get_transform(train=True))
dataset_valid = CatDogDataset(df['new_path'], df['new_bb'], df['class'], get_transform(train=False))

In [293]:
torch.manual_seed(1)
indices = torch.randperm(len(dataset_train)).tolist()

dataset_train = Subset(dataset_train, indices[:-50])
dataset_valid = Subset(dataset_valid, indices[-50:])

In [294]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [295]:
dl_train = DataLoader(dataset_train, 
                      batch_size=2, 
                      shuffle=True, 
                      num_workers=4, 
                      collate_fn=collate_fn)

dl_valid = DataLoader(dataset_valid, 
                      batch_size=1, 
                      shuffle=False, 
                      num_workers=4, 
                      collate_fn=collate_fn)

In [298]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2

model = CatsDogsNet()

model.to(device)

optimizer = optim.Adam(model.parameters())

lr_scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                         step_size=3, 
                                         gamma=0.1)

In [300]:
num_epochs = 10

for epoch in range(num_epochs):
    
    train_one_epoch(model, optimizer, dl_train, device, epoch, print_freq=1)
    
    # Update the learning rate
    lr_scheduler.step()
    
    evaluate(model, dl_valid, device=device)

KeyboardInterrupt: 